In [16]:
import mysql.connector

# Database connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="arbazatm"
)
cursor = conn.cursor()

# Create tables if they don't exist
create_table_query_0 = """
CREATE TABLE IF NOT EXISTS atm_db (
    ACCOUNT_HOLDER_NAME VARCHAR(30),
    ACCOUNT_NO INT PRIMARY KEY,
    PASSWORD INT
)
"""
cursor.execute(create_table_query_0)

create_table_query_1 = """
CREATE TABLE IF NOT EXISTS transaction_db (
    ACCOUNT_NO INT,
    TRANSACTION_ID INT PRIMARY KEY AUTO_INCREMENT,
    WITHDRAW_AMOUNT INT DEFAULT 0,
    DEPOSIT_AMOUNT INT DEFAULT 0,
    CURRENT_BALANCE INT DEFAULT 0,
    FOREIGN KEY (ACCOUNT_NO) REFERENCES atm_db(ACCOUNT_NO)
)
"""
cursor.execute(create_table_query_1)

def insert_query_for_atm():
    ACCOUNT_HOLDER_NAME = input("ENTER FULL NAME: ").strip()
    ACCOUNT_NO = int(input("ENTER ACCOUNT NUMBER: "))
    PASSWORD = int(input("ENTER PASSWORD: "))
    
    try:
        # Insert into the atm_db table
        insert_query = """
        INSERT INTO atm_db (ACCOUNT_HOLDER_NAME, ACCOUNT_NO, PASSWORD)
        VALUES (%s, %s, %s)
        """
        cursor.execute(insert_query, (ACCOUNT_HOLDER_NAME, ACCOUNT_NO, PASSWORD))
        
        # Initialize the transaction record for the new user in transaction_db
        init_transaction = """
        INSERT INTO transaction_db (ACCOUNT_NO, CURRENT_BALANCE)
        VALUES (%s, %s)
        """
        cursor.execute(init_transaction, (ACCOUNT_NO, 0))
        
        conn.commit()
        print("Account created successfully!")
    except mysql.connector.Error as err:
        print(f"Error: {err}")


# Function to check current balance
def current_balance():
    select_query = "SELECT CURRENT_BALANCE FROM transaction_db WHERE ACCOUNT_NO = %s"
    cursor.execute(select_query, (USER,))
    balance = cursor.fetchone()
    if balance:
        print(f"Current Balance: {balance[0]}")
    else:
        print("No account found!")

# Function to deposit an amount
def deposit_amount():
    deposit = int(input("ENTER AMOUNT TO DEPOSIT: "))
    if deposit <= 0:
        print("Deposit amount must be positive!")
        return

    # Update the deposit and current balance
    update_deposit_query = """
    UPDATE transaction_db
    SET DEPOSIT_AMOUNT = DEPOSIT_AMOUNT + %s, CURRENT_BALANCE = CURRENT_BALANCE + %s
    WHERE ACCOUNT_NO = %s
    """
    cursor.execute(update_deposit_query, (deposit, deposit, USER))
    conn.commit()
    print(f"{deposit} deposited successfully!")
    current_balance()

# Function to withdraw an amount
def withdraw_amount():
    withdraw = int(input("ENTER AMOUNT TO WITHDRAW: "))
    if withdraw <= 0:
        print("Withdrawal amount must be positive!")
        return

    # Check current balance
    select_balance_query = "SELECT CURRENT_BALANCE FROM transaction_db WHERE ACCOUNT_NO = %s"
    cursor.execute(select_balance_query, (USER,))
    balance = cursor.fetchone()
    if balance and balance[0] >= withdraw:
        # Update the withdrawal and current balance
        update_withdraw_query = """
        UPDATE transaction_db
        SET WITHDRAW_AMOUNT = WITHDRAW_AMOUNT + %s, CURRENT_BALANCE = CURRENT_BALANCE - %s
        WHERE ACCOUNT_NO = %s
        """
        cursor.execute(update_withdraw_query, (withdraw, withdraw, USER))
        conn.commit()
        print(f"{withdraw} withdrawn successfully!")
        current_balance()
    else:
        print("Insufficient balance!")

# Main program
USER = int(input("ENTER ACCOUNT NUMBER: "))
PASSW = int(input("ENTER PASSWORD: "))
select_query = "SELECT * FROM atm_db WHERE ACCOUNT_NO = %s AND PASSWORD = %s"
cursor.execute(select_query, (USER, PASSW))
LOGIN_TRY = cursor.fetchone()

if LOGIN_TRY:
    print("\nLOGIN SUCCESSFUL!")
    while True:
        print("\nCHOOSE AN OPTION:\n1. CHECK CURRENT BALANCE\n2. DEPOSIT AMOUNT\n3. WITHDRAW AMOUNT\n4. EXIT")
        choice = input("ENTER YOUR CHOICE: ")
        if choice == "1":
            current_balance()
        elif choice == "2":
            deposit_amount()
        elif choice == "3":
            withdraw_amount()
        elif choice == "4":
            print("Thank you for using our service!")
            break
        else:
            print("INVALID OPTION! TRY AGAIN.")
else:
    print("INVALID ACCOUNT NUMBER OR PASSWORD!")


ENTER ACCOUNT NUMBER:  1234
ENTER PASSWORD:  1234



LOGIN SUCCESSFUL!

CHOOSE AN OPTION:
1. CHECK CURRENT BALANCE
2. DEPOSIT AMOUNT
3. WITHDRAW AMOUNT
4. EXIT


ENTER YOUR CHOICE:  1


Current Balance: 0

CHOOSE AN OPTION:
1. CHECK CURRENT BALANCE
2. DEPOSIT AMOUNT
3. WITHDRAW AMOUNT
4. EXIT


ENTER YOUR CHOICE:  2
ENTER AMOUNT TO DEPOSIT:  90000


90000 deposited successfully!
Current Balance: 90000

CHOOSE AN OPTION:
1. CHECK CURRENT BALANCE
2. DEPOSIT AMOUNT
3. WITHDRAW AMOUNT
4. EXIT


ENTER YOUR CHOICE:  100


INVALID OPTION! TRY AGAIN.

CHOOSE AN OPTION:
1. CHECK CURRENT BALANCE
2. DEPOSIT AMOUNT
3. WITHDRAW AMOUNT
4. EXIT


ENTER YOUR CHOICE:  4


Thank you for using our service!


In [15]:
insert_query_for_atm()

ENTER FULL NAME:  Arbaz
ENTER ACCOUNT NUMBER:  1234
ENTER PASSWORD:  1234


Account created successfully!
